# Projeto 2 - Como pegar dados de um site com Python? - Pegando dados de ETFs do mundo inteiro.


### Desafio:

* Construir um código que vá no site etf.com e busque dados de todos os etfs do mercado americano e, consequentemente, do mundo. Rentabilidades, patrimônio, gestora, taxa...   
* Lembrar de sempre trazer outros tipos de cenários onde a pessoa precisa pegar dados de sites etc.


### Passo a passo:

   **Passo 1** - Definir um navegador que você irá utilizar para navegar com o Python.

   **Passo 2** - Importar os módulos e bibliotecas.
   
   **Passo 3** - Entender como funcionam requisições na internet.
   
   **Passo 4** - Conhecer e mapear o processo de coleta de dados no site do ETF.com. 
   
   **Passo 5** - Achar todos os elementos necessários dentro do HTML do site.
   
   **Passo 6** - Ler a tabela de dados.
   
   **Passo 7** - Construir a tabela final.

In [ ]:
!pip install webdriver-manager

In [ ]:
!pip install selenium

In [ ]:
!pip install html5lib

# Passo 1: Escolher o navegador.

No nosso caso, utilizaremos o Google Chrome. 

# Passo 2: Importar as bibliotecas.

In [21]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

# Passo 3: Entender como funcionam requisições na internet.

In [22]:
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()))

url = "https://www.etf.com/etfanalytics/etf-finder"

driver.get(url)

# Passo 4: Conhecer e mapear o processo de coleta de dados no site do ETF.com.

www.etf.com

# Processo de dados

* Abrir o site - Completo
* Mudar a visualização pra 100
* Ler a tabela
* Avançar todas as páginas
* Trocar pra outra categoria
* Ler todas as tabelas dessa outra categoria

# Passo 5.1: Achar todos os elementos necessários dentro do HTML do site - Expandindo a tabela para 100 itens.

In [25]:
time.sleep(5)

botao_100 = driver.find_element("xpath", '''/html/body/div[4]/section/div/div[3]/
section/div/div/div/div/div[2]/section[2]/
div[2]/section[2]/div[1]/div/div[4]/button/label/span''')

driver.execute_script("arguments[0].click();", botao_100)

# Passo 5.2: Achar todos os elementos necessários dentro do HTML do site - Pegando o número de páginas da tabela.

In [26]:
numero_paginas = driver.find_element("xpath", '''/html/body/div[4]/section/div/div[3]/
section/div/div/div/div/div[2]/section[2]/
div[2]/section[2]/div[2]/div/label[2]''')

numero_paginas = numero_paginas.text.replace("of ", "")

numero_paginas = int(numero_paginas)

print(numero_paginas)

32


# Passo 6.1: Lendo a tabela de dados - Lendo a tabela de dados básicos.

In [27]:
lista_de_tabela_por_pagina = []

for pagina in range(0, numero_paginas):
    
    tabela = driver.find_element("xpath", '''/html/body/div[4]/section/div/
    div[3]/section/div/div/div/
    div/div[2]/section[2]/div[2]/div/table''')

    html_tabela = tabela.get_attribute("outerHTML")

    tabela_final = pd.read_html(html_tabela)[0]
    
    lista_de_tabela_por_pagina.append(tabela_final)
    
    botao_avancar_pagina = driver.find_element("xpath", '//*[@id="nextPage"]')
    
    driver.execute_script("arguments[0].click();", botao_avancar_pagina)
    

base_de_dados_completa = pd.concat(lista_de_tabela_por_pagina)

display(base_de_dados_completa)

[WDM] - Downloading: 100%|██████████| 8.72M/8.72M [57:10<00:00, 2.66kB/s]


,Ticker,Name,Segment,Issuer,Expense Ratio,AUM
0,ZHDG,ZEGA Buy and Hedge ETF,Asset Allocation: U.S. Target Outcome,Tidal,0.98%,$56.76M
1,ZECP,Zacks Earnings Consistent Portfolio ETF,Equity: U.S. - Total Market,Zacks,0.55%,$27.44M
2,TSLY,YieldMax TSLA Option Income Strategy ETF,Equity: U.S. Automobile Manufacturers,Tidal,0.99%,$825.16K
3,OARK,YieldMax Innovation Option Income Strategy ETF,Equity: Global Broad Thematic,Tidal,0.99%,$456.33K
4,HYLB,Xtrackers USD High Yield Corporate Bond ETF,"Fixed Income: U.S. - Corporate, Broad-based Hi...",DWS,0.15%,$4.02B
...,...,...,...,...,...,...
8,PFLD,AAM Low Duration Preferred and Income Securiti...,"Fixed Income: U.S. - Corporate, Preferred Shor...",Advisors Asset Management,0.45%,$199.35M
9,SMIG,AAM Bahl & Gaynor Small/Mid Cap Income Growth ETF,Equity: U.S. - Extended Market,Advisors Asset Management,0.60%,$185.02M
10,PRNT,3D Printing ETF,Equity: Global Robotics & AI,ARK,0.66%,$191.27M
11,EGIS,2ndVote Society Defended ETF,Equity: U.S. - Total Market,"2nd Vote Value Investments, Inc",0.75%,$25.81M


# Passo 6.2: Ler a tabela de dados - Lendo a tabela de dados de rentabilidade.

In [32]:
#primeira coisa é mudar a aba

botao_aba = driver.find_element("xpath", ''' /html/body/div[4]/
section/div/div[3]/section/
div/div/div/div/div[2]/section[2]/div[2]/ul/li[2]/span''')

driver.execute_script("arguments[0].click();", botao_aba)

#voltando a 1 página

for pagina in range(0, numero_paginas):
    
    botao_voltar_pagina = driver.find_element("xpath", '//*[@id="previousPage"]')
    
    driver.execute_script("arguments[0].click();", botao_voltar_pagina)

In [35]:
lista_de_tabela_por_pagina = []

for pagina in range(0, numero_paginas):
    
    tabela = driver.find_element("xpath", '''/html/body/div[4]/section/div/
    div[3]/section/div/div/div/
    div/div[2]/section[2]/div[2]/div/table''')

    html_tabela = tabela.get_attribute("outerHTML")

    tabela_final = pd.read_html(html_tabela)[0]
    
    lista_de_tabela_por_pagina.append(tabela_final)
    
    botao_avancar_pagina = driver.find_element("xpath", '//*[@id="nextPage"]')
    
    driver.execute_script("arguments[0].click();", botao_avancar_pagina)
    

base_de_dados_performance = pd.concat(lista_de_tabela_por_pagina)

display(base_de_dados_performance.head())

,Ticker,Name,1 Month,3 Month,YTD,1 Year,3 Years,5 Years,10 Years,As Of Date
0,PALL,Aberdeen Standard Physical Palladium Shares ETF,-8.78%,-13.10%,-9.22%,-29.25%,-11.80%,9.99%,7.30%,02/09/23
1,PPLT,abdrn Physical Platinum Shares ETF,-11.73%,-3.35%,-11.05%,-8.34%,-1.06%,-0.76%,-6.26%,02/09/23
2,YEAR,AB Ultra Short Income ETF,0.16%,1.06%,0.36%,--,--,--,--,02/09/23
3,TAFI,AB Tax-Aware Short Duration Municipal ETF,0.32%,2.36%,0.68%,--,--,--,--,02/09/23
4,TRFM,AAM Transformers ETF,13.78%,23.64%,17.66%,--,--,--,--,02/09/23


In [36]:
driver.quit()

In [ ]:
base_de_dados_performance

# Passo 7: Construindo a tabela final.

In [42]:
base_de_dados_completa = base_de_dados_completa.set_index("Ticker")

base_de_dados_completa

KeyError: "None of ['Ticker'] are in the columns"

In [41]:
base_de_dados_performance = base_de_dados_performance.set_index("Ticker")
base_de_dados_performance = base_de_dados_performance[['1 Year', '5 Years', '10 Years']]

base_de_dados_performance

KeyError: "None of ['Ticker'] are in the columns"

In [43]:
base_de_dados_final = base_de_dados_completa.join(base_de_dados_performance)

In [63]:
display(base_de_dados_completa)
display(base_de_dados_performance)

,Name,Segment,Issuer,Expense Ratio,AUM
Ticker,,,,,
LYFE,2ndVote Life Neutral Plus ETF,Equity: U.S. - Total Market,"2nd Vote Value Investments, Inc",0.75%,$17.54M
EGIS,2ndVote Society Defended ETF,Equity: U.S. - Total Market,"2nd Vote Value Investments, Inc",0.75%,$26.10M
PRNT,3D Printing ETF,Equity: Global Robotics & AI,ARK,0.66%,$187.71M
SMIG,AAM Bahl & Gaynor Small/Mid Cap Income Growth ETF,Equity: U.S. - Extended Market,Advisors Asset Management,0.60%,$173.39M
PFLD,AAM Low Duration Preferred and Income Securiti...,"Fixed Income: U.S. - Corporate, Preferred Shor...",Advisors Asset Management,0.45%,$197.08M
...,...,...,...,...,...
HYLB,Xtrackers USD High Yield Corporate Bond ETF,"Fixed Income: U.S. - Corporate, Broad-based Hi...",DWS,0.15%,$4.14B
OARK,YieldMax Innovation Option Income Strategy ETF,Equity: Global Broad Thematic,Tidal,0.99%,$483.93K
TSLY,YieldMax TSLA Option Income Strategy ETF,Equity: U.S. Automobile Manufacturers,Tidal,0.99%,$386.59K


,1 Year,5 Years,10 Years
Ticker,,,
LYFE,-1.80%,--,--
EGIS,2.38%,--,--
PRNT,-28.09%,-3.51%,--
SMIG,-3.34%,--,--
PFLD,-8.24%,--,--
...,...,...,...
HYLB,-5.99%,2.33%,--
OARK,--,--,--
TSLY,--,--,--


In [44]:
base_de_dados_final

,Name,Segment,Issuer,Expense Ratio,AUM,1 Year,5 Years,10 Years
Ticker,,,,,,,,
AAA,AXS First Priority CLO Bond ETF,"Fixed Income: U.S. - Corporate, Bank Loans Inv...",AXS Investments,0.25%,$7.40M,NaN,NaN,NaN
AAAU,Goldman Sachs Physical Gold ETF,Commodities: Precious Metals Gold,Goldman Sachs,0.18%,$605.89M,NaN,NaN,NaN
AADR,AdvisorShares Dorsey Wright ADR ETF,Equity: Global Ex-U.S. - Total Market,AdvisorShares,1.00%,$29.54M,NaN,NaN,NaN
AAPB,GraniteShares 1.75x Long AAPL Daily ETF,Leveraged Equity: U.S. Technology Hardware Sto...,GraniteShares,1.15%,$998.02K,NaN,NaN,NaN
AAPD,Direxion Daily AAPL Bear 1X Shares,Inverse Equity: U.S. Technology Hardware Stora...,Direxion,0.97%,$22.58M,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
ZIG,The Acquirers Fund,Equity: U.S. - Large Cap Value,ETF Series Solutions,0.89%,$39.03M,NaN,NaN,NaN
ZROZ,PIMCO 25+ Year Zero Coupon US Treasury Index ETF,"Fixed Income: U.S. - Government, Treasury Inve...",PIMCO,0.15%,$822.00M,NaN,NaN,NaN
ZSB,USCF Sustainable Battery Metals Strategy Fund,Commodities: Broad Market Metals,Marygold,0.59%,$2.54M,NaN,NaN,NaN


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

url = 'https://www.etf.com/etfanalytics/etf-finder'

driver.get(url)

time.sleep(5)

cem_resultados = driver.find_element("xpath", 
                    '''/html/body/div[5]/section/div/div[3]/section/div/
                    div/div/div/div[2]/section[2]/div[2]/section[2]/div[1]/div/div[4]/button/label/span''')

driver.execute_script("arguments[0].click();", cem_resultados)

time.sleep(2)

numero_paginas = driver.find_element("xpath", '''//*[@id="totalPages"]''')

numero_paginas = numero_paginas.text.replace("of ", "")

numero_paginas = int(numero_paginas)

lista_data_frames_por_pagina = []

for pagina in range(0, numero_paginas):

    local_tabela = '''
    //*[@id="finderTable"]
    '''

    elemento = driver.find_element("xpath", local_tabela)

    html_tabela = elemento.get_attribute('outerHTML')

    tabela = pd.read_html(html_tabela)[0]
    
    lista_data_frames_por_pagina.append(tabela)
    
    if pagina != numero_paginas:
    
        botao_avancar_pag = driver.find_element("xpath", '//*[@id="nextPage"]')

        driver.execute_script("arguments[0].click();", botao_avancar_pag)


    

base_dados_etf_cadastro = pd.concat(lista_data_frames_por_pagina)

#voltando tudo

for pagina in range(0, numero_paginas):

    botao_voltar_pag = driver.find_element("xpath", '//*[@id="previousPage"]')

    driver.execute_script("arguments[0].click();", botao_voltar_pag)

# a unica coisa que muda é esse botão
botao_dados_perf = driver.find_element("xpath", '/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/ul/li[2]/span')


# a partir daqui é tudo igual
driver.execute_script("arguments[0].click();", botao_dados_perf)

lista_data_frames_por_pagina = []

for pagina in range(0, numero_paginas):


    local_tabela = '''
    //*[@id="finderTable"]
    '''

    elemento = driver.find_element("xpath", local_tabela)

    html_tabela = elemento.get_attribute('outerHTML')

    tabela = pd.read_html(html_tabela)[0]
    
    lista_data_frames_por_pagina.append(tabela)
    
    if pagina != numero_paginas:
    
        botao_avancar_pag = driver.find_element("xpath", '//*[@id="nextPage"]')

        driver.execute_script("arguments[0].click();", botao_avancar_pag)
    

base_dados_etf_performance = pd.concat(lista_data_frames_por_pagina)

#fechando navegador
driver.quit()

base_dados_etf_cadastro = base_dados_etf_cadastro.set_index('Ticker')
base_dados_etf_performance = base_dados_etf_performance.set_index("Ticker")
base_dados_etf_performance = base_dados_etf_performance[['1 Year', '5 Years', '10 Years']]

base_dados_final = base_dados_etf_cadastro.join(base_dados_etf_performance, how = "inner")

base_dados_final